Install Dependencies

In [55]:
# Processing libraries
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

#Imports pickle
import pickle

#Library for indicators
from finta import TA

#Another library for indicators
from ta.utils import dropna
from ta.volatility import BollingerBands
from ta.trend import PSARIndicator
from ta.trend import SMAIndicator
from ta.trend import macd
from ta.momentum import rsi
from ta.volume import OnBalanceVolumeIndicator

#Import yfinance
import yfinance as yf

Selects the dataframe for training

In [110]:
#Downloads dataframe from internet

df = yf.download(tickers="BTC-USD",
                            period="15y",
                            interval="1d",
                            auto_adjust=True)

[*********************100%***********************]  1 of 1 completed


In [93]:

#Loads dataframe from folder

#df = pd.read_csv('/content/drive/MyDrive/Other Projects/USDGBP.csv')
#df = pd.read_csv('/content/drive/MyDrive/Other Projects/USDJPY.csv')
#df = pd.read_csv('/Users/esteban/Documents/GitHub/Reinforcement_Learning_Binary_Options_Bot/BTCUSD_1hr.csv')
#df = pd.read_csv('BTCUSD_day.csv')
#df = pd.read_csv('BTCUSD_1hr.csv')
#df = pd.read_csv('/content/drive/MyDrive/Other Projects/BTCUSD_1hr.csv')
#df = pd.read_csv('/content/drive/MyDrive/Other Projects/BTCUSD_1min.csv')
#df = pd.read_csv('/content/drive/MyDrive/Other Projects/Download Data - STOCK_US_XNYS_F (1).csv')


Data Cleanup

In [104]:
#print(df.dtypes)

#Convert date
#df['Date'] = pd.to_datetime(df['Date'])

#Sort values from past to present
#df.sort_values('Date', ascending=True, inplace=True)

#Set date as index
#df.set_index('Date', inplace=True)

#Converts volume from string to float
#df.drop('Volume', inplace=True, axis=1)
#df['Volume'] = df['Volume'].apply(lambda x: float(x.replace(",", "")))
#df['Volume'] = df['Volume'].astype(float)


#Preparation for hourly df
#df = df.drop(columns=['Symbol'])
#df = df.drop(columns=['Unix Timestamp'])
#df = df.drop(columns=['Volume'])

#Preparation for downloaded df

df = df.reset_index(drop=True)
df = df.drop(columns=['Volume'])

'''
#Preparation for daily dataset
df = df.drop(columns=['Volume USD'])
df = df.rename(columns={'Volume BTC': 'Volume'})
'''

print(df.shape[0])
print(df.dtypes)

df.head()

1753
Open     float64
High     float64
Low      float64
Close    float64
dtype: object


,Open,High,Low,Close
0,308.644989,329.451996,307.056000,320.884003
1,320.670990,324.717987,294.541992,299.252991
2,298.585999,319.453003,298.191986,314.681000
3,314.690002,319.153015,298.513000,307.907990
4,307.024994,328.415009,307.024994,316.716003


2.1.3. Calculate SMA, RSI and OBV


In [111]:

#Indicators with TA
#Adds simple moving average
df["sma7"] = SMAIndicator(close=df["Open"], window=7, fillna=True).sma_indicator()
df["sma25"] = SMAIndicator(close=df["Open"], window=25, fillna=True).sma_indicator()
df["sma99"] = SMAIndicator(close=df["Open"], window=99, fillna=True).sma_indicator()

#Adds Bollinger Bands
indicator_bb = BollingerBands(close=df["Open"], window=20, window_dev=2)
df['bb_bbm'] = indicator_bb.bollinger_mavg()
df['bb_bbh'] = indicator_bb.bollinger_hband()
df['bb_bbl'] = indicator_bb.bollinger_lband()

#Adds PSAR Indicator
indicator_psar = PSARIndicator(high=df["Close"], low=df["Low"], close=df["Close"], step=0.02, max_step=2, fillna=True)
df['psar'] = indicator_psar.psar()

# Add Moving Average Convergence Divergence (MACD) indicator
df["MACD"] = macd(close=df["Close"], window_slow=26, window_fast=12, fillna=True)
    
# Add Relative Strength Index (RSI) indicator
df["RSI"] = rsi(close=df["Close"], window=14, fillna=True)

#Indicators using finta
#Adds simple moving average
df["sma40"] = TA.SMA(df, period = 40)

# Add Relative Strength Index (RSI) indicator
#df["OBV"] = TA.OBV(df)

#Drops empty spaces
df.fillna(0, inplace=True)

#Drops empty spaces
df.head(50)


/Users/esteban/miniforge3/envs/tf_m1/lib/python3.8/site-packages/ta/trend.py:938: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  self._psar_up = pd.Series(index=self._psar.index)
/Users/esteban/miniforge3/envs/tf_m1/lib/python3.8/site-packages/ta/trend.py:939: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  self._psar_down = pd.Series(index=self._psar.index)


,Open,High,Low,Close,Volume,sma7,sma25,sma99,bb_bbm,bb_bbh,bb_bbl,psar,MACD,RSI,sma40
Date,,,,,,,,,,,,,,,
2014-09-17,465.864014,468.174011,452.421997,457.334015,21056800,465.864014,465.864014,465.864014,0.000000,0.000000,0.000000,457.334015,0.000000,100.000000,0.000000
2014-09-18,456.859985,456.859985,413.104004,424.440002,34483200,461.362000,461.362000,461.362000,0.000000,0.000000,0.000000,424.440002,-2.624024,0.000000,0.000000
2014-09-19,424.102997,427.834991,384.532013,394.795990,37919700,448.942332,448.942332,448.942332,0.000000,0.000000,0.000000,457.334015,-7.014744,0.000000,0.000000
2014-09-20,394.673004,423.295990,389.882996,408.903992,36863600,435.375000,435.375000,435.375000,0.000000,0.000000,0.000000,455.877975,-9.249402,20.155072,0.000000
2014-09-21,408.084991,412.425995,393.181000,398.821014,26580100,429.916998,429.916998,429.916998,0.000000,0.000000,0.000000,454.451056,-11.699137,17.448333,0.000000
2014-09-22,399.100006,406.915985,397.130005,402.152008,24127600,424.780833,424.780833,424.780833,0.000000,0.000000,0.000000,453.052675,-13.219400,21.212666,0.000000
2014-09-23,402.092010,441.557007,396.196991,435.790985,45099500,421.539573,421.539573,421.539573,0.000000,0.000000,0.000000,451.682262,-11.576391,47.332023,0.000000
2014-09-24,435.751007,436.112000,421.131989,423.204987,30627700,417.237714,423.316002,423.316002,0.000000,0.000000,0.000000,450.339257,-11.161219,41.754527,0.000000
2014-09-25,423.156006,423.519989,409.467987,411.574005,26814400,412.422860,423.298225,423.298225,0.000000,0.000000,0.000000,449.023112,-11.636576,37.371826,0.000000


In [112]:
#add a column to dataframe
df.drop(df.tail(1).index,inplace=True)
df['Position'] = np.where((df['Open']-df['Close'].shift(-1)) < 1, 1, 0)

#df["Position"] = df['Open'] - df['Close'].shift(-1)



In [113]:
df = df.reset_index(drop=True)


Dump dataset into a pickle file

In [108]:
import pickle
with open("BTCUSD_10y_tagged.pickle", 'wb') as f:
      pickle.dump(df,f, pickle.HIGHEST_PROTOCOL)
